# それぞれのスレのレスを取得
スレタイにキーワード該当がある場合はThreadsに全レスを記録  
レスにキーワード該当がある場合はResponsesにそのレスを記録（上記の場合も含む）  
途中で中断しても、全実行で続きからスクレイピングすることができる  
リセットしたい場合はProgresses/Threadsにあるファイルを消去←このファイルが消去されている場合はスクレイピング結果がリセットされて最初からやりなおされるため注意

In [1]:
#データ管理のID
ID = "mimikkyu"

#リンク保存するファイル
links_file = "Links/%s.txt"%ID

#スレ保存するファイル
threads_file = "Threads/%s.txt"%ID

#レス保存するファイル
responses_file = "Responses/%s.txt"%ID

#スレタイを保存するファイル
titles_file = "Titles/%s.txt"%ID

#進捗ファイル
progress_file = "Progresses/Threads/%s.txt"%ID

#キーワードファイル
keywords_file = "Keywords/%s.txt"%ID

#無条件で保存するか（母集団取得のため）
GET_POPULATION = True
population_file = "Responses/%s-population.txt"%(ID)

#並列数
WORKERS_N = 30

#スクレイピングするファイル数（全スレ抽出する場合はNoneに）
WANTED_THREADS = 50000

## 進捗を確認し、スクレイピングする順番を決定

In [ ]:
import random

random.seed(334)

In [ ]:
#対象リストを取得
with open(links_file, "r", encoding="utf-8") as f:
    links = f.read().split()

#進捗を確認
processed = []
try:
    with open(progress_file, "r", encoding="utf-8") as f:
        processed = list(map(int, f.read().split()))
except:
    #進捗ファイルがない（初回）
    with open(progress_file, "w", encoding="utf-8") as f:
        processed = []

#順番を決定
order = []
for cnt in range(len(links)):
    if cnt not in processed:
        #未スクレイピング
        order.append(cnt)

#シャッフル
random.shuffle(order)

len(processed), len(order)

In [ ]:
#ランダム抽出
if WANTED_THREADS != None:
    order = random.sample(order, min(WANTED_THREADS, len(order)))

len(order)

## スレ・レス・スレタイファイルを用意
  
スレ・レスの形式は  
Y/M/D|H/M/S <\t> リンクインデックス <\t> レス番号 <\t> レス内容  
  
スレタイの形式は  
リンクインデックス <\t> スレタイ
　

In [ ]:
if len(processed) == 0:
    with open(threads_file, "w", encoding="utf-8") as f:
        f.write("date\tlink_index\tnumber\tcontent\n")

    with open(responses_file, "w", encoding="utf-8") as f:
        f.write("date\tlink_index\tnumber\tcontent\n")

    with open(population_file, "w", encoding="utf-8") as f:
        f.write("date\tlink_index\tnumber\tcontent\n")
    
    with open(titles_file, "w", encoding="utf-8") as f:
        f.write("link_index\ttitle\n")

## キーワードを取得

In [2]:
keywords = []
with open(keywords_file) as f:
    keywords = f.read().split()

keywords

['ミミッキュ']

## スクレイピング

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary
import concurrent.futures
from threading import Lock
from time import sleep
import datetime
import re

### Seleniumブラウザを用意する

In [ ]:
drivers = []

class Driver:
    def __init__(self, option):
        self.driver = webdriver.Chrome(options=option)
        self.used = False

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
for _ in range(WORKERS_N):
    drivers.append(Driver(chrome_options))

### 指定のページからスクレイピング

#### 安価先の取得

In [5]:
s = " >>1 >>5 >>76 キキョウシティのBGMすこ"

def get_ankers(text):
    result = re.findall(r"\>\>\d+", text, re.S)
    
    numbers = []
    for anker in result:
        numbers.append(str(anker[2:]))

    return set(numbers)

get_ankers(s)

{'1', '5', '76'}

#### 投稿内容の処理 

In [ ]:
def process_content(content):
    result = content.replace("\t", "<\\t>").replace("\n", "<\\br>")
    return result

#### 各URLに対する処理

In [ ]:
#textにキーワードがあったら
def has_keyword(text):
    for keyword in keywords:
        if keyword in text:
            return True
    return False

#返り値：メタ情報、コンテント
def get_from_current(soup):
    thread_div = soup.find("div", class_="thread")
    posts_div = thread_div.find_all("div", class_="post")

    metas = []
    contents = []

    for post_div in posts_div:
        #メタ情報
        number = post_div.find("span", class_="number").text
        
        date = post_div.find("span", class_="date").text
        date = get_time(date)
        
        metas.append((number, date))

        #内容
        content = post_div.find("div", class_="message").text
        contents.append(content)

    return (metas, contents)

def get_from_past(soup):
    #投稿内容
    dd = soup.find_all("dd")
    dd[0].find("div").decompose()

    #メタ情報
    dt = soup.find_all("dt")

    metas = []
    contents = []
    for cnt in range(len(dd)):
        #メタ情報
        meta = str(dt[cnt])
        #投稿番号
        result = re.match(r'<dt>\d+', meta, re.S)
        if result != None:
            number = result.group(0).split("<dt>")[1]
        else:
            number = None

        #投稿時間
        date = get_time(meta)
        
        metas.append((number, date))

        #投稿内容
        content = dd[cnt].text      
        contents.append(content)

    return metas, contents  

def get_time(string):
    try:
        #日付
        result = re.search(r"\d{4}\/\d{2}\/\d{2}", string, re.S)
        if result == None:
            result = re.search(r"\d{2}\/\d{2}\/\d{2}", string, re.S)
            date_string = "20"+result.group(0)
        else:
            date_string = result.group(0)

        #時間
        result = re.search(r"\d{2}\:\d{2}", string, re.S)

        time_string = result.group(0)

        date = datetime.datetime.strptime(date_string+"|"+time_string, "%Y/%m/%d|%H:%M")
  
        return date
    except:
        return None

In [ ]:
#ファイル記入重複防止
write_lock = Lock()

#レス、スレファイルへの記入
def write_response(f, time, thread_index, response_index, content):
    if time != None:
        _time = time.strftime("%Y/%m/%d|%H/%M/%S")
    else:
        _time = None
    f.write(str(_time)+"\t"
            +str(thread_index)+"\t"
            +str(response_index)+"\t"
            +content+"\n")

def write_titles(f, link_index, title):
    f.write(str(link_index)+"\t"
            +title+"\n")

#使われていないブラウザを返す
def GetUnusedDriver():
    global drivers
    
    for driver in drivers:
        if driver.used == False:
            return driver
    
    print("ALL USED")

def scrape(link, link_index):
    global write_lock
    #ブラウザを使い始める
    driver = GetUnusedDriver()
    if driver != None:
        driver.used = True
    else:
        #ブラウザがなかった
        return
    
    try:
        #アクセス
        while True:
            driver.driver.get(link)

            sleep(0.5)

            html = driver.driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            #人大杉対策
            title = soup.find("title").text
            
            if ("error" in title):
                #時間をおいてやり直し
                sleep(1)
            else:
                #アクセス完了
                break

        #現在型か過去型か判別
        if soup.find_all("meta")[1].get("property") == "og:title":
            current = False
        else:
            current = True

        #タイトルにキーワードがある場合はThreadsに全レスを記入
        threads = has_keyword(title)
        
        #スクレイピング
        if current:
            metas, contents = get_from_current(soup)
        else:
            metas, contents = get_from_past(soup)

        #同時書き込み防止
        with write_lock:
            #処理済みを記入
            with open(progress_file, "a", encoding="utf-8") as f:
                f.write(str(link_index) + "\n")

            #タイトル記入
            with open(titles_file, "a", encoding="utf-8") as f:
                write_titles(f, link_index, title)

            #スレ・レス記入
            n = len(metas)
            found_keyword = []
            
            with open(threads_file, "a", encoding="utf-8") as threads_f:
                with open(responses_file, "a", encoding="utf-8") as responses_f:
                    with open(population_file, "a", encoding="utf-8") as population_f:
                        for cnt in range(n):
                            content = process_content(contents[cnt])
                            number, time = metas[cnt]

                            #母集団記録モード
                            if GET_POPULATION:
                                write_response(population_f, time, link_index, number, content)

                            #スレへの記入
                            if threads:
                                write_response(threads_f, time, link_index, number, content)

                            #レスへの記入
                            #レスにキーワードがあったら or 安価先に言及済みがあったら
                            if has_keyword(content) | (len(get_ankers(content) & set(found_keyword)) > 0):
                                found_keyword.append(number) 
                                write_response(responses_f, time, link_index, number, content)

    except Exception as e:
        print(link)
        print(str(e))

    #使い終わった
    driver.used = False

## 並列スクレイピング

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=WORKERS_N) as executor:
    futures = [executor.submit(scrape, links[index], index) for index in order]

    #完了まで待つ
    _ = concurrent.futures.as_completed(fs = futures)